In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from polygon import RESTClient
import numpy as np


In [2]:
EURPLN = pd.read_csv("EURPLN.csv", parse_dates=['datetime'])
GBPCHF = pd.read_csv("GBPCHF.csv", parse_dates=['datetime'])
HKDJPY = pd.read_csv("HKDJPY.csv", parse_dates=['datetime'])
NZDUSD = pd.read_csv("NZDUSD.csv", parse_dates=['datetime'])
SGDJPY = pd.read_csv("SGDJPY.csv", parse_dates=['datetime'])
USDBRL = pd.read_csv("USDBRL.csv", parse_dates=['datetime'])
USDDKK = pd.read_csv("USDDKK.csv", parse_dates=['datetime'])
USDHKD = pd.read_csv("USDHKD.csv", parse_dates=['datetime'])
USDMXN = pd.read_csv("USDMXN.csv", parse_dates=['datetime'])
USDTRY = pd.read_csv("USDTRY.csv", parse_dates=['datetime'])


/var/folders/0b/j_4ztbtn3jv7jxjvy_fpqgrc0000gn/T/ipykernel_1774/2343674708.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  EURPLN = pd.read_csv("EURPLN.csv", parse_dates=['datetime'])
/var/folders/0b/j_4ztbtn3jv7jxjvy_fpqgrc0000gn/T/ipykernel_1774/2343674708.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  NZDUSD = pd.read_csv("NZDUSD.csv", parse_dates=['datetime'])
/var/folders/0b/j_4ztbtn3jv7jxjvy_fpqgrc0000gn/T/ipykernel_1774/2343674708.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  USDDKK = pd.read_csv("USDDKK.csv", parse_dates=['datetime'])


In [3]:
def resample_data(df, start_date, end_date):
    # Filter data by date range
    data = df[(df['datetime'] >= start_date) & (df['datetime'] < end_date)]

    data['price_times_transactions'] = data['vw'] * data['n']

    # Group the data into the specified frequency intervals
    grouped_data = data.groupby(pd.Grouper(key='datetime', freq='6H'))

    # Calculate VWAP and Liquidity for each group
    grouped_vwap = grouped_data['price_times_transactions'].sum() / grouped_data['n'].sum()
    grouped_liquidity = grouped_data['n'].sum() / 6


    grouped_vwap = grouped_vwap.fillna(0)
    max_value = grouped_data['h'].max().fillna(0)
    min_value = grouped_data['l'].min().fillna(0)

    # Combine the results into a new DataFrame
    resampled_df = pd.DataFrame({
        'timestamp': grouped_vwap.index,
        'vwap': grouped_vwap.values,
        'liquidity': grouped_liquidity.values,
        'max': max_value,
        'min': min_value
    })

    resampled_df['volatility'] = (resampled_df['max'] - resampled_df['min']) / resampled_df['vwap']

    return resampled_df


In [4]:
start_date = "2022-01-01"
end_date = "2023-01-01"
EURPLN = resample_data(EURPLN, start_date, end_date)
GBPCHF = resample_data(GBPCHF, start_date, end_date)
HKDJPY = resample_data(HKDJPY, start_date, end_date)
NZDUSD = resample_data(NZDUSD, start_date, end_date)
SGDJPY = resample_data(SGDJPY, start_date, end_date)
USDBRL = resample_data(USDBRL, start_date, end_date)
USDDKK = resample_data(USDDKK, start_date, end_date)
USDHKD = resample_data(USDHKD, start_date, end_date)
USDMXN = resample_data(USDMXN, start_date, end_date)
USDTRY = resample_data(USDTRY, start_date, end_date)

/var/folders/0b/j_4ztbtn3jv7jxjvy_fpqgrc0000gn/T/ipykernel_1774/3558701289.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price_times_transactions'] = data['vw'] * data['n']
/var/folders/0b/j_4ztbtn3jv7jxjvy_fpqgrc0000gn/T/ipykernel_1774/3558701289.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price_times_transactions'] = data['vw'] * data['n']
/var/folders/0b/j_4ztbtn3jv7jxjvy_fpqgrc0000gn/T/ipykernel_1774/3558701289.py:5: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [5]:
def calculate_fd_values(df):
    fd_values = [0]
    for idx in range(1, len(df)):
        max_rate = df.iloc[idx - 1]['max']
        min_rate = df.iloc[idx - 1]['min']
        vol = df.iloc[idx - 1]['volatility']
        mean_rate = df.iloc[idx - 1]['vwap']

        keltner_upper = [mean_rate + n * 0.025 * vol for n in range(1, 101)]
        keltner_lower = [mean_rate - n * 0.025 * vol for n in range(1, 101)]

        prices = df.iloc[idx - 1:idx + 1]['vwap'].values

        crosses = calculate_fd(prices, keltner_upper, keltner_lower)

        fd = 0
        if (max_rate - min_rate) != 0:
            fd = crosses / (max_rate - min_rate)
        elif crosses == 0:
            fd = 0
        else:
            fd = 0

        fd_values.append(fd)

    df['fd'] = fd_values
    return df


def calculate_fd(prices, keltner_upper, keltner_lower):
    crosses = 0
    for i in range(1, len(prices)):
        for upper, lower in zip(keltner_upper, keltner_lower):
            if (prices[i - 1] < upper and prices[i] > upper) or (prices[i - 1] > lower and prices[i] < lower):
                crosses += 1
    return crosses

In [6]:
EURPLN = calculate_fd_values(EURPLN)
GBPCHF = calculate_fd_values(GBPCHF)
HKDJPY = calculate_fd_values(HKDJPY)
NZDUSD = calculate_fd_values(NZDUSD)
SGDJPY = calculate_fd_values(SGDJPY)
USDBRL = calculate_fd_values(USDBRL)
USDDKK = calculate_fd_values(USDDKK)
USDHKD = calculate_fd_values(USDHKD)
USDMXN = calculate_fd_values(USDMXN)
USDTRY = calculate_fd_values(USDTRY)

In [12]:
EURPLN.dropna(inplace=True)
GBPCHF.dropna(inplace=True)
HKDJPY.dropna(inplace=True)
NZDUSD.dropna(inplace=True)
SGDJPY.dropna(inplace=True)
USDBRL.dropna(inplace=True)
USDDKK.dropna(inplace=True)
USDHKD.dropna(inplace=True)
USDMXN.dropna(inplace=True)
USDTRY.dropna(inplace=True)

In [16]:
SGDJPY.isnull().values.any()

False

In [20]:
USDDKK

,timestamp,vwap,liquidity,max,min,volatility,fd
datetime,,,,,,,
2022-01-02 18:00:00,2022-01-02 18:00:00,6.538475,221.666667,6.54220,6.53392,0.001266,0.000000
2022-01-03 00:00:00,2022-01-03 00:00:00,6.551173,1732.333333,6.56070,6.53633,0.003720,12077.294686
2022-01-03 06:00:00,2022-01-03 06:00:00,6.553903,2893.333333,6.56047,6.54120,0.002940,1189.987690
2022-01-03 12:00:00,2022-01-03 12:00:00,6.570143,5033.500000,6.59352,6.54381,0.007566,5189.413596
2022-01-03 18:00:00,2022-01-03 18:00:00,6.583883,1929.500000,6.59142,6.57894,0.001896,1448.400724
...,...,...,...,...,...,...,...
2022-12-30 06:00:00,2022-12-30 06:00:00,6.974032,5707.666667,6.98985,6.95068,0.005617,1695.842451
2022-12-30 12:00:00,2022-12-30 12:00:00,6.963663,7279.833333,6.98527,6.94170,0.006257,1863.671177
2022-12-30 18:00:00,2022-12-30 18:00:00,6.947010,2463.833333,6.95455,6.93980,0.002123,2295.157218


In [45]:
pip install pandas pymongo dnspython


     |████████████████████████████████| 413 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 283 kB 25.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [46]:
from pymongo import MongoClient

In [47]:
client = MongoClient("mongodb://localhost:27017/")


In [48]:
db = client["fx_pairs"]


In [49]:
collection = db["USDDKK"]

In [50]:
data_to_insert = USDDKK.to_dict("records")

In [52]:
result = collection.insert_many(data_to_insert)


ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 643ae31f1d08ba9b101bc94f, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


X = USDDKK[['vwap', 'volatility', 'liquidity']]
y = USDDKK['fd']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


model = LinearRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [37]:
mse

6161273987.058567

In [38]:
r2

-0.0035008331756496425

In [44]:
# Initialize the PyCaret setup for regression
from pycaret.regression import *
USDDKK_setup = setup(data = USDDKK, target='vwap', train_size=0.7)

# X = resampled_df[['vwap', 'liquidity', 'max', 'min', 'volatility']]
# y = resampled_df['fd']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


OSError: dlopen(/Users/jinchengbaby/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <D21A7969-4567-3BC7-94ED-6A9E83AE9D78> /Users/jinchengbaby/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

In [ ]:
# from pycaret.regression import *

# # Initialize the regression module
# reg = setup(data= , target='fd')

# # Compare different regression models
# compare_models()

# # Train a model
# model = create_model('lr')

# # Evaluate the model on the test set
# predict_model(model, data=X_test)


In [42]:
# Initialize the models
from sklearn.linear_model import Ridge, Lasso, ElasticNet

ridge = Ridge(alpha=1.0)
lasso = Lasso(alpha=1.0)
elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5)

models = {
    'Ridge Regression': ridge,
    'Lasso Regression': lasso,
    'Elastic Net Regression': elastic_net
}

# Train and evaluate the models
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f'{model_name}:')
    print(f'Mean Squared Error: {mse:.2f}')
    print(f'R^2 Score: {r2:.5f}\n')

Ridge Regression:
Mean Squared Error: 6161273987.06
R^2 Score: -0.00350

Lasso Regression:
Mean Squared Error: 6161273992.09
R^2 Score: -0.00350

Elastic Net Regression:
Mean Squared Error: 6161273989.61
R^2 Score: -0.00350

